In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import helper
import postgres


In [3]:
# file_path = "../../Data/Global_Mobility_Report.csv"
file_path = helper.download_file(url='https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv')


Global_Mobility_Report.csv: 100%|##########| 971M/971M [05:25<00:00, 2.99Mit/s] 


In [4]:
postgres.create_tables_google()

df = helper.pd.read_csv(
    filepath_or_buffer=file_path, parse_dates=True, low_memory=False
)


Creating connection to the PostgreSQL database...
Table countries_google already exists! Skipping...
Table location_util_google already exists! Skipping...
Table mobility_stats_google already exists! Skipping...


In [5]:
res = helper.last_entry("mobility_stats_google")
df = df[df["date"] > str(res["date"].values[0])]


Creating connection to the PostgreSQL database...


In [6]:
countries = df.loc[:, ("country_region_code", "country_region")].drop_duplicates()
countries["country_region_code"][countries["country_region"] == "Namibia"] = "NA"
countries = countries.sort_values(by="country_region_code").reset_index(
    drop=True
)

stats = df.drop(
    columns=["country_region", "iso_3166_2_code", "census_fips_code", "place_id"]
)

location_util = (
    df.loc[
        :, ("country_region_code", "iso_3166_2_code", "census_fips_code", "place_id")
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)


In [7]:
query = "select * from countries_google"
existing_countries = (
    pd.read_sql_query(sql=query, con=helper._make_connection().connect())
    .sort_values(by="country_region_code", axis=0)
    .reset_index(drop=True)
)


Creating connection to the PostgreSQL database...


In [8]:
countries = [
    x
    for x in countries.values
    if x[0] not in existing_countries["country_region_code"].values
]
countries = pd.DataFrame(countries, columns=["country_region_code", "country_region"])


In [9]:
query = "select * from location_util_google"
existing_util = pd.read_sql_query(sql=query, con=helper._make_connection().connect())


Creating connection to the PostgreSQL database...


In [10]:
location_util = [
    x
    for x in location_util.values
    if x[0] not in existing_util.country_region_code.values
]
location_util = pd.DataFrame(
    location_util,
    columns=["country_region_code", "iso_3166_2_code", "census_fips_code", "place_id"],
)


In [11]:
postgres.import_data(countries, "countries_google")
postgres.import_data(stats, "mobility_stats_google")
postgres.import_data(location_util, "location_util_google")


Creating connection to the PostgreSQL database...
Creating connection to the PostgreSQL database...
Creating connection to the PostgreSQL database...
